In [1]:
# import required libraries
import requests
from bs4 import BeautifulSoup
import unicodedata
import pandas as pd

In [2]:
# States present in v-mart retail
states = ["Andhra Pradesh","Arunachal Pradesh","Assam","Bihar","Chandigarh","Delhi","Goa","Gujarat","Haryana",
          "Himachal Pradesh","Jammu and Kashmir","Jharkhand","Karnataka","Kerala","Madhya Pradesh","Maharashtra",
          "Meghalaya",
          "Nagaland","Odisha","Puducherry","Punjab","Rajasthan","Tamil Nadu","Telangana","Tripura","Uttar Pradesh",
          "Uttarakhand","West Bengal"]

In [3]:
# vmart retail url link
url = "https://stores.vmartretail.com/location/"
header = {'User-Agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19577"}


In [4]:
address_dict = {}

for state in states:
    
    temp_url = url + state
    r = requests.get(temp_url , headers = header)
    soup = BeautifulSoup(r.text , "html.parser")
    title = soup.find_all(class_="card h-100 info-card idx-str-info-card rounded-0")
    
    store_detail_list = []

    for class_title in title :
        store_details = {}
        # store name
        store_details['store_name'] = unicodedata.normalize('NFKD', class_title.find(class_= "idx-info-card-str-name").get_text())
        # store address
        store_details['address'] = class_title.find(class_= "store-address-info idx-info-card-str-add-info").get_text()
        # store opening and closing time 
        store_details['open_time'], store_details['close_time'] = class_title.find(class_= "store-time-info idx-info-card-str-time-info").get_text().split(' to ')
        # store coordinates(latitude and longitude)
        map_url = class_title.find(class_= "get_direction_click" , href = True)['href']
        lat , long = list(map(float , map_url.split('=')[-1].split(',')))
        
        store_details['latitude'] = lat
        store_details['longitude'] = long
        store_details['contact'] = class_title.find(class_= "store-contact-info idx-info-card-str-cont-info").get_text()
        
        store_detail_list.append(store_details)
        
    address_dict[state] = store_detail_list

In [5]:
data_list = []
for state in states:
    temp_data = pd.json_normalize(address_dict[state])
#     temp_data['state'] = state
    data_list.append(temp_data)
data = pd.concat(data_list, ignore_index=True)

In [6]:
data

,store_name,address,open_time,close_time,latitude,longitude,contact
0,"V-Mart Narsipatnam, Visakhapatnam","Bank Colony, Main Road, Narsipatnam, Visakhap...",11:00 AM,10:00 PM,17.662204,82.620501,0971 7966 511
1,"V-Mart Peda Madina, Visakhapatnam","Door No 48/14/92/1 And 93, Gnt Road, Cmr Road...",11:00 AM,10:00 PM,17.737511,82.834711,0928 9062 315
2,"V-Mart Rajahmundry, East Godavari","Door No 79/1/1, Rs No 288/1, Jn Road, Mallina...",11:00 AM,10:00 PM,16.972893,81.833365,0928 9062 312
3,"V-Mart Eluru, West Godavari","Plot No 453/1, 433/9A1 And 451, Pathebad Road...",11:00 AM,10:00 PM,16.718211,81.119511,0928 9062 310
4,"V-Mart Vijayawada, Krishna","Shop No 40/1/44, Ground 1St And 2Nd Floor, Ms...",11:00 AM,10:00 PM,16.515099,80.632095,0928 9062 311
...,...,...,...,...,...,...,...
304,"V-Mart Uttarpara, Hooghly","Bandhan Banquet, Hindmotor, Gt Road, Ghospara...",10:00 AM,10:00 PM,22.683027,88.347774,0704 2989 922
305,"V-Mart Ghospara, Uttarpara","Bandhan Banquet, Hindmotor, GT Road, Ghospara...",10:00 AM,10:00 PM,22.683886,88.354086,0704 2989 922
306,"V-Mart Esplanade, Kolkata","Shop No 7, Metropolitan Building, Jawahar Lal...",11:00 AM,10:00 PM,22.563549,88.351913,0956 0622 886
307,"V-Mart Sethpukhar, Barasat","65/6, Jessore Road N, Sethpukhar, Sethpukhar,...",10:00 AM,10:00 PM,22.715939,88.483142,0742 8495 283


In [7]:
data.to_csv('V-mart_stores')

In [8]:
pd.read_csv('V-mart_stores')

,Unnamed: 0,store_name,address,open_time,close_time,latitude,longitude,contact
0,0,"V-Mart Narsipatnam, Visakhapatnam","Bank Colony, Main Road, Narsipatnam, Visakhap...",11:00 AM,10:00 PM,17.662204,82.620501,0971 7966 511
1,1,"V-Mart Peda Madina, Visakhapatnam","Door No 48/14/92/1 And 93, Gnt Road, Cmr Road...",11:00 AM,10:00 PM,17.737511,82.834711,0928 9062 315
2,2,"V-Mart Rajahmundry, East Godavari","Door No 79/1/1, Rs No 288/1, Jn Road, Mallina...",11:00 AM,10:00 PM,16.972893,81.833365,0928 9062 312
3,3,"V-Mart Eluru, West Godavari","Plot No 453/1, 433/9A1 And 451, Pathebad Road...",11:00 AM,10:00 PM,16.718211,81.119511,0928 9062 310
4,4,"V-Mart Vijayawada, Krishna","Shop No 40/1/44, Ground 1St And 2Nd Floor, Ms...",11:00 AM,10:00 PM,16.515099,80.632095,0928 9062 311
...,...,...,...,...,...,...,...,...
304,304,"V-Mart Uttarpara, Hooghly","Bandhan Banquet, Hindmotor, Gt Road, Ghospara...",10:00 AM,10:00 PM,22.683027,88.347774,0704 2989 922
305,305,"V-Mart Ghospara, Uttarpara","Bandhan Banquet, Hindmotor, GT Road, Ghospara...",10:00 AM,10:00 PM,22.683886,88.354086,0704 2989 922
306,306,"V-Mart Esplanade, Kolkata","Shop No 7, Metropolitan Building, Jawahar Lal...",11:00 AM,10:00 PM,22.563549,88.351913,0956 0622 886
307,307,"V-Mart Sethpukhar, Barasat","65/6, Jessore Road N, Sethpukhar, Sethpukhar,...",10:00 AM,10:00 PM,22.715939,88.483142,0742 8495 283
